In [1]:
import os
from ames.harvesters import file_mapping, get_usage, build_usage
from ames.harvesters import build_aggregate, aggregate_usage
from ames.harvesters import get_caltechdata, get_history
from ames.matchers import add_usage, submit_report

if os.path.isdir("data") == False:
    os.mkdir("data")
os.chdir("data")

production = True
collection = "caltechdata.ds"

get_caltechdata(collection, production)
mapping = file_mapping(collection)



100% (3 of 3) |##########################| Elapsed Time: 0:00:11 Time:  0:00:11
100% (2030 of 2030) |####################| Elapsed Time: 0:00:22 Time:  0:00:22


In [6]:
import requests

downloads = {}

days = 1
while days < 31:
    response = requests.get(f'https://stats.tind.io/index.php?date=2021-12-{days:02}&expanded=1&filter_limit=-1&format=JSON&idSite=1161&method=Actions.getDownloads&module=API&period=day&segment=&token_auth=TOKEN')
    for dl in response.json()[0]['subtable']:
        unique = dl['nb_uniq_visitors']
        idv = 'https://data.caltech.edu'+ dl['label']
        if idv in mapping:
            label = mapping[idv]
            if label in downloads:
                downloads[label] = downloads[label] + unique
            else:
                downloads[label] = unique
    days = days + 1
        
#for d in downloads:
#    print(d)
#    print(downloads[d])

In [7]:
from ames.harvesters import get_records
from py_dataset import dataset

dot_paths = ['._Key','.publicationYear','.electronic_location_and_access']
keys = dataset.keys(collection)
all_metadata = get_records(dot_paths,'dois',collection,keys)

rec_by_pub_year = {}
dl_by_pub_year = {}

for record in all_metadata:
    key = record['_Key']
    year = record['publicationYear']
    if 'electronic_location_and_access' in record:
        if year in rec_by_pub_year:
            rec_by_pub_year[year] += 1
        else:
            rec_by_pub_year[year] = 1
    if key in downloads:
        if year in dl_by_pub_year:
            dl_by_pub_year[year] += 1
        else:
            dl_by_pub_year[year] = 1

print(rec_by_pub_year['2017'])
print(rec_by_pub_year['2018'])
print(rec_by_pub_year['2019'])
print(rec_by_pub_year['2020'])
print(rec_by_pub_year['2021'])
print(dl_by_pub_year['2017']/rec_by_pub_year['2017'])
print(dl_by_pub_year['2018']/rec_by_pub_year['2018'])
print(dl_by_pub_year['2019']/rec_by_pub_year['2019'])
print(dl_by_pub_year['2020']/rec_by_pub_year['2020'])
print(dl_by_pub_year['2021']/rec_by_pub_year['2021'])        

49
134
149
420
367
0.08163265306122448
0.029850746268656716
0.11409395973154363
0.08571428571428572
0.10626702997275204
